In [1]:
#我们尝试取出前x个词向量，组成一个二维矩阵，当做图片进行CNN的训练

In [1]:
import numpy as np
import pandas as pd
import jieba
from gensim.models import word2vec
olderr = np.seterr(all='ignore')
import os

C:\Users\ty020\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
f = open("D:\\杂七杂八课程\\CUMCM-template-note\\NLP\\doubanMoviereview--master\\scrapy_data\\data_long\\三傻大闹宝莱坞.csv")
raw_data = pd.read_csv(f,usecols=['评论内容', '推荐星数'])
raw_data
raw_labels = raw_data["推荐星数"].tolist()
y_labels = []
for label in raw_labels:
    if label == "很差":
        y_labels.append(1)
    elif label == "较差":
        y_labels.append(2)
    elif label == "还行":
        y_labels.append(3)
    elif label == "推荐":
        y_labels.append(4)
    else:
        y_labels.append(5)
x_comments = raw_data["评论内容"].tolist()
positive_comments = []
negative_comments = []
tmp_comments = []
doc_words = []
for label,comment in zip(y_labels, x_comments):
    if(label == 1):
        result = list(jieba.cut(comment,cut_all=False))
        positive_comments += result
        doc_words.append(result)
    else:
        result = list(jieba.cut(comment.strip(), cut_all=False))
        negative_comments += result
        doc_words.append(result)
for i in range(len(positive_comments)):
    if positive_comments[i] not in "、，“”。？！；：...1234567890‘’-《》 ":
        tmp_comments.append(positive_comments[i])
positive_comments = tmp_comments
tmp_comments = []
for i in range(len(negative_comments)):
    if negative_comments[i] not in "、，“”。？！；：...1234567890‘’-《》 ":
        tmp_comments.append(negative_comments[i])
negative_comments = tmp_comments
len(doc_words)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ty020\AppData\Local\Temp\jieba.cache
Loading model cost 0.581 seconds.
Prefix dict has been built succesfully.


3609

In [3]:
comments = positive_comments + negative_comments
model=word2vec.Word2Vec(doc_words, size=256, min_count=1,hs=0)

In [4]:
for i in range(3609):
    tmp = []
    for word in doc_words[i]:
        if word not in "、，“”。？！；：...1234567890‘’-《》 ":
            tmp.append(word)
    doc_words[i] = tmp

In [58]:
# 我们对每个评论随机抽取20个单词，每个单词维度是256
doc_matrix = []
len_words = 40
for i in range(len(doc_words)):
    matrix = []
    if len(doc_words[i]) >= len_words:
        words = np.array(doc_words[i])
        choose = np.array(words[:20])
        for word in choose:
            matrix.append(np.array(model.wv[word]))
    else:
        for word in doc_words[i]:
            matrix.append(np.array(model.wv[word]))
        for i in range(len(doc_words[i]),len_words):
            matrix.append(np.zeros(256))
    doc_matrix.append(np.array(matrix))

In [59]:
print(np.array(doc_matrix).shape)

(3609, 20, 256)


In [27]:
#split train and split
from sklearn.model_selection import train_test_split
train_comments, test_comments, train_labels, test_labels = train_test_split(np.array(doc_matrix).reshape((3609, 20, 256, 1))
                                                                            , y_labels, test_size=0.33, random_state=42)


ValueError: cannot reshape array of size 72180 into shape (3609,20,256,1)

In [60]:
train_comments.shape

(2418, 20, 256, 1)

In [61]:
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
train_labels_done = keras.utils.to_categorical(train_labels, 6)
cnn_model = tf.keras.Sequential([
    keras.layers.Conv2D(32,(3,3), data_format='channels_last', input_shape=(20,256,1),activation="relu"),
    keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),
    keras.layers.Dropout(rate=0.2,seed=1),
    keras.layers.Conv2D(32,(3,3),activation="relu"),
    keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),
    keras.layers.Dropout(rate=0.2,seed=1),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(6, activation="softmax"),
    ])

cnn_model.compile(optimizer=tf.train.AdamOptimizer(), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
print(cnn_model.summary())
with tf.device('/cpu:0'):
    cnn_model.fit(train_comments, train_labels_done, batch_size=32, epochs=20)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 18, 254, 32)       320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 127, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 9, 127, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 125, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 63, 32)         0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 4, 63, 32)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8064)              0         
__________

In [62]:
test_labels_done = keras.utils.to_categorical(test_labels, 6)
test_loss, test_acc = cnn_model.evaluate(np.array(test_comments), test_labels_done)

1191/1191 [==============================] - 1s 554us/step


In [63]:
test_loss, test_acc

(1.1161583589466553, 0.6666666667667585)